In [69]:
import sys
import argparse
import warnings
from pathlib import Path
from scripts.processing import Processor
from scripts.folder_structure import FolderStructure
from scripts.analyze import load_pyseer_hits
from scripts.analyze import AnalyzeResults
from scripts.figures import get_k_loci, FIGURE4_PANELA, FIGURE4_PANELB, FIGURE4_PANELC


%load_ext autoreload
%autoreload 2

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', 100)

# workflow IO
input_dir = "/Users/januszkoszucki/MGG Dropbox/Projects/kleb-prophage-div/2025-02-12_KLEBDATA_LIGHT"
output_dir = "/Users/januszkoszucki/MGG Dropbox/Projects/gwas/2025-03-26_KLEBDATA_LIGHT_GWAS"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [70]:
# folder structure
folder_manager = FolderStructure(input_dir, output_dir)
structure = folder_manager.get_paths_dict()
params = folder_manager.get_params_dict()

# aggregate results
processor = Processor(structure, params)
processor.aggregate_pyseer_hits()
processor.aggregate_function_prediction()

# load
pyseer_hits = load_pyseer_hits(structure, params)

Created directory: /Users/januszkoszucki/tmp_blast_dir


In [71]:
analyze = AnalyzeResults(structure, params)

analyze.per_locus_analysis(run=False)
analyze.aggregate_data(run=False)
analyze.combine_predictions_and_enzymes(run=False)

In [77]:
FIGURE4_PANELA(structure, params)
# FIGURE4_PANELB(structure, params)
# FIGURE4_PANELC(structure, params)

In [128]:
# paths
cluster_functions_table = '/Users/januszkoszucki/MGG Dropbox/Projects/gwas/2025-03-26_KLEBDATA_LIGHT_GWAS/1_INTERMEDIATE/3_FUNCTIONS/HHSEARCH/PCI50C50/clusters_functions.tsv'
cluster_map_table = '/Users/januszkoszucki/MGG Dropbox/Projects/gwas/2025-03-26_KLEBDATA_LIGHT_GWAS/1_INTERMEDIATE/3_FUNCTIONS/HHSEARCH/PCI50C50/clusters_map.tsv'

# load
cluster_functions_df = pd.read_csv(cluster_functions_table, sep='\t')
pcs = list(cluster_functions_df.loc[cluster_functions_df['reported_topology_PC'] == 'no hit']['PC'].unique())

hits_pc = pyseer_hits.query('mode =="lasso" and version == "PCI50C50" and precision_SC >= 0.75 and recall_SC >= 0.1')
pcs = list(hits_pc['PC'].unique())
cluster_functions_no_hit_df = cluster_functions_df.query('PC.isin(@pcs)').query('reported_topology_PC == "no hit"')
cluster_functions_no_hit_df.to_csv('/Users/januszkoszucki/Downloads/no_hit.tsv', sep='\t', index=False)

'|'.join(cluster_functions_no_hit_df['PC'].unique())

'PC0467|PC0585|PC0712|PC1255|PC1259|PC1260|PC1270|PC1458|PC1500|PC1503|PC1869'

In [131]:
another_list_pcs = list(cluster_functions_no_hit_df['PC'].unique())
pyseer_hits.query('mode =="lasso" and version == "PCI50C50" and precision_SC >= 0.75 and recall_SC >= 0.1 and PC.isin(@another_list_pcs)')[['PC', 'locus', 'precision_SC', 'recall_SC']]

,PC,locus,precision_SC,recall_SC
4212,PC0712,KL111,0.88,0.50
4272,PC1255,KL14,1.00,0.27
4273,PC1869,KL14,1.00,0.18
4382,PC1270,KL25,1.00,0.40
4410,PC0467,KL3,1.00,0.75
4419,PC0585,KL30,1.00,0.17
4472,PC1259,KL47,1.00,0.36
4473,PC1260,KL47,1.00,0.36
4474,PC1500,KL47,1.00,0.36
4492,PC1458,KL6,0.75,0.17
